**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_numerical -p 8888:8888 -d palmoreck/jupyterlab_numerical:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_numerical
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

---

Nota generada a partir de [liga](https://drive.google.com/file/d/12L7rOCgW7NEKl_xJbIGZz05XXVrOaPBz/view).

In [91]:
!pip3 install --user -q cvxpy

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Constrained Equality Convex Optimization (CECO)

Se comparan los resultados del **paquete [cvxpy](https://github.com/cvxgrp/cvxpy)** con los obtenidos en la implementación hecha por el prof en [algoritmos/Python](algoritmos/Python), en específico [algoritmos/Python/algorithms_for_ceco.py](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/IV.optimizacion_convexa_y_machine_learning/algoritmos/Python/algorithms_for_ceco.py) para problemas tipo CECO (*Constrained Equality Convex Optimization*)

In [1]:
import os

In [2]:
cur_directory = os.getcwd()

In [3]:
dir_alg_python = '/algoritmos/Python'

In [4]:
os.chdir(cur_directory + dir_alg_python)

In [5]:
import math

import numpy as np

import algorithms_for_ceco
from line_search import line_search_for_residual_by_backtracking
from utils import compute_error


# Primer ejemplo

$$\displaystyle \min_{x \in \mathbb{R}^2} \quad x_1^2 + 2x_1x_2 + x_2^2-2x_2$$

$$\text{sujeto a: } x_1 = 0$$

In [6]:
fo = lambda x: x[0]**2 + 2*x[0]*x[1]+x[1]**2-2*x[1]

In [7]:
A = np.array([1,0],dtype=float)

In [8]:
b = np.array([0])

In [9]:
x_ast=np.array([0,1], dtype=float)

## Punto inicial $x^{(0)}$ factible ($Ax^{(0)}=b$)

In [10]:
x_0 = np.array([0,-2],dtype=float)

In [11]:
tol=1e-8
tol_backtracking=1e-14
maxiter=50
p_ast=fo(x_ast)
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_feasible_init_point(fo,A, x_0,tol, 
                                                                         tol_backtracking, x_ast, p_ast, maxiter)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	7.21e+00	1.80e+01	3.00e+00	9.00e+00	---		9.00e+03
1	7.21e+00	1.15e-05	2.40e-03	5.76e-06	1.00e+00	9.00e+03
2	7.21e+00	7.55e-15	6.68e-08	4.44e-15	1.00e+00	9.00e+03
Error of x with respect to x_ast: 6.68e-08
Approximate solution: [4.33680869e-19 9.99999933e-01]


In [12]:
x

array([4.33680869e-19, 9.99999933e-01])

In [13]:
total_of_iterations

3

In [14]:
x_plot.shape

(2, 3)

In [15]:
x_plot

array([[ 0.00000000e+00,  0.00000000e+00,  4.33680869e-19],
       [-2.00000000e+00,  1.00239973e+00,  9.99999933e-01]])

In [16]:
compute_error(x_ast,x)

6.682210107467057e-08

## En el siguiente ejemplo si usamos el algoritmo anterior para punto inicial no factible ($Ax^{(0)} \neq b$) obsérvese que no funciona

In [17]:
x_0 = np.array([1,-2],dtype=float)

In [18]:
tol=1e-8
tol_backtracking=1e-14
maxiter=50
p_ast=fo(x_ast)
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_feasible_init_point(fo,A, x_0,tol, 
                                                                         tol_backtracking, x_ast, p_ast, maxiter)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	4.47e+00	8.01e+00	3.16e+00	6.00e+00	---		2.03e+07
1	4.47e+00	5.12e-06	1.41e+00	2.00e+00	1.00e+00	2.03e+07
2	4.47e+00	9.86e-16	1.41e+00	2.00e+00	1.00e+00	2.03e+07
Error of x with respect to x_ast: 1.41e+00
Approximate solution: [ 1.00000000e+00 -3.34455919e-08]


In [19]:
compute_error(x_ast,x)

1.4142135860226999

**Error relativo muy alto**

## Método de Newton para puntos iniciales no factibles

### Punto inicial no factible ($Ax^{(0)} \neq b$)

In [20]:
x_0 = np.array([1,-2], dtype=float)
nu_0 = 5

**Versión del método de Newton para puntos iniciales no factibles que se apoya del método de Newton para puntos iniciales factibles**

In [21]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast, maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.00e+00	5.00e+00	1.00e+01	3.16e+00	6.00e+00	---		2.03e+07
1	5.00e-01	2.50e+00	3.00e+00	1.58e+00	2.00e+00	5.00e-01	2.03e+07
2	2.50e-01	1.25e+00	9.99e-01	7.90e-01	7.49e-01	5.00e-01	2.03e+07
3	1.25e-01	6.25e-01	3.75e-01	3.95e-01	3.12e-01	5.00e-01	2.03e+07
4	6.25e-02	3.12e-01	1.56e-01	1.98e-01	1.41e-01	5.00e-01	2.03e+07
5	3.13e-02	1.56e-01	7.03e-02	9.88e-02	6.64e-02	5.00e-01	2.03e+07
6	1.56e-02	7.81e-02	3.32e-02	4.94e-02	3.22e-02	5.00e-01	2.03e+07
7	7.81e-03	3.90e-02	1.61e-02	2.47e-02	1.59e-02	5.00e-01	2.03e+07
8	3.91e-03	1.95e-02	7.93e-03	1.23e-02	7.87e-03	5.00e-01	2.03e+07
9	1.95e-03	9.76e-03	3.94e-03	6.17e-03	3.92e-03	5.00e-01	2.03e+07
10	9.77e-04	4.88e-03	1.96e-03	3.09e-03	1.96e-03	5.00e-01	2.03e+07
11	4.88e-04	2.44e-03	9.78e-04	1.54e-03	9.78e-04	5.00e-01	2.03e+07
12	2.44e-04	1.22e-03	4.89e-04	7.72e-04	4.89e-04	5.00e-01	2.03e+07
13	1.22e-04	6.10e-04	2.44e-04	3.86e-04	2.44e-04	5.00e-01	2.03e+07

In [22]:
compute_error(x_ast,x)

2.554210325052966e-08

**Versión del método de Newton para puntos iniciales no factibles (2o versión)**

In [23]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point_2nd_version(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast, maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.00e+00	5.00e+00	1.00e+01	3.16e+00	6.00e+00	---		2.03e+07
1	5.00e-01	2.50e+00	3.00e+00	1.58e+00	2.00e+00	5.00e-01	2.03e+07
2	2.50e-01	1.25e+00	9.99e-01	7.90e-01	7.49e-01	5.00e-01	2.03e+07
3	1.25e-01	6.25e-01	3.75e-01	3.95e-01	3.12e-01	5.00e-01	2.03e+07
4	6.25e-02	3.12e-01	1.56e-01	1.98e-01	1.41e-01	5.00e-01	2.03e+07
5	3.13e-02	1.56e-01	7.03e-02	9.88e-02	6.64e-02	5.00e-01	2.03e+07
6	1.56e-02	7.81e-02	3.32e-02	4.94e-02	3.22e-02	5.00e-01	2.03e+07
7	7.81e-03	3.90e-02	1.61e-02	2.47e-02	1.59e-02	5.00e-01	2.03e+07
8	3.91e-03	1.95e-02	7.93e-03	1.23e-02	7.87e-03	5.00e-01	2.03e+07
9	1.95e-03	9.76e-03	3.94e-03	6.17e-03	3.92e-03	5.00e-01	2.03e+07
10	9.77e-04	4.88e-03	1.96e-03	3.09e-03	1.96e-03	5.00e-01	2.03e+07
11	4.88e-04	2.44e-03	9.78e-04	1.54e-03	9.78e-04	5.00e-01	2.03e+07
12	2.44e-04	1.22e-03	4.89e-04	7.72e-04	4.89e-04	5.00e-01	2.03e+07
13	1.22e-04	6.10e-04	2.44e-04	3.86e-04	2.44e-04	5.00e-01	2.03e+07

In [24]:
compute_error(x_ast,x)

1.1430297930381528e-08

# Segundo ejemplo

$$\displaystyle \min_{x \in \mathbb{R}^2} \quad \frac{1}{2}(x_1^2 + x_2^2)$$

$$\text{sujeto a: } 2x_1 -x_2 = 5$$

In [25]:
fo = lambda x: 1/2*(x[0]**2  + x[1]**2)

In [26]:
A = np.array([2,-1],dtype=float)

In [27]:
b=5

In [28]:
x_ast = np.array([2,-1],dtype=float)

## Punto inicial $x^{(0)}$ factible ($Ax^{(0)}=b$)

In [29]:
x_0 = np.array([0,-5],dtype=float)

In [30]:
tol=1e-8
tol_backtracking=1e-14
maxiter=50
p_ast=fo(x_ast)
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_feasible_init_point(fo,A, x_0,tol, 
                                                                         tol_backtracking, x_ast, p_ast, maxiter)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	5.00e+00	2.00e+01	2.00e+00	4.00e+00	---		1.00e+00
1	5.00e+00	3.94e-06	8.88e-04	7.89e-07	1.00e+00	1.00e+00
2	5.00e+00	3.56e-12	8.63e-07	7.45e-13	1.00e+00	1.00e+00
Error of x with respect to x_ast: 8.63e-07
Approximate solution: [ 1.99999914 -1.00000173]


In [31]:
x

array([ 1.99999914, -1.00000173])

In [32]:
x_ast

array([ 2., -1.])

In [33]:
compute_error(x_ast,x)

8.631750624932266e-07

## Método de Newton para puntos iniciales no factibles

### Punto inicial no factible ($Ax^{(0)} \neq b$)

In [34]:
x_0 = np.array([1,1],dtype=float)
nu_0 = 4

**Versión del método de Newton para puntos iniciales no factibles que se apoya del método de Newton para puntos iniciales factibles**

In [35]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast, maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	4.00e+00	9.49e+00	1.00e+00	1.00e+00	6.00e-01	---		1.00e+00
1	0.00e+00	7.56e-04	3.19e-08	7.99e-05	6.38e-09	1.00e+00	1.00e+00
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
Beginning Newtons method for feasible initial point
I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	2.24e+00	3.19e-08	7.99e-05	6.38e-09	---		1.00e+00
1	2.24e+00	9.87e-15	4.08e-08	1.60e-15	1.00e+00	1.00e+00
Error of x with respect to x_ast: 4.08e-08
Approximate solution: [ 1.99999996 -1.00000008]


In [36]:
compute_error(x_ast,x)

4.078981343802468e-08

**Versión del método de Newton para puntos iniciales no factibles (2o versión)**

In [37]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point_2nd_version(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast, maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	4.00e+00	9.49e+00	1.00e+00	1.00e+00	6.00e-01	---		1.00e+00
1	0.00e+00	7.56e-04	3.19e-08	7.99e-05	6.38e-09	1.00e+00	1.00e+00
2	0.00e+00	6.58e-08	3.55e-15	6.20e-09	1.78e-16	1.00e+00	1.00e+00
3	8.88e-16	1.03e-09	8.88e-16	1.29e-08	1.78e-16	2.50e-01	1.00e+00
Error of x with respect to x_ast: 1.29e-08
Approximate solution: [ 1.99999999 -1.00000003]


In [38]:
compute_error(x_ast,x)

1.2867310816844223e-08

# Tercer ejemplo

$$\displaystyle \min_{x \in \mathbb{R}^2} \quad x_1^2 + x_2^2$$

$$\text{sujeto a :} x_1+x_2 = 1$$

In [39]:
fo = lambda x: x[0]**2+x[1]**2

In [40]:
x_ast = np.array([.5,.5],dtype=float)

In [41]:
A = np.array([1,1],dtype=float)

In [42]:
b=1

## Punto inicial $x^{(0)}$ factible ($Ax^{(0)}=b$)

In [43]:
x_0 = np.array([2,-1],dtype=float)

In [44]:
tol=1e-8
tol_backtracking=1e-14
maxiter=50
p_ast=fo(x_ast)
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_feasible_init_point(fo,A, x_0,tol, 
                                                                         tol_backtracking, x_ast, p_ast, maxiter)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	4.47e+00	9.00e+00	3.00e+00	9.00e+00	---		1.00e+00
1	4.47e+00	7.11e-08	2.67e-04	7.11e-08	1.00e+00	1.00e+00
2	4.47e+00	1.81e-16	2.46e-12	1.11e-16	1.00e+00	1.00e+00
Error of x with respect to x_ast: 2.46e-12
Approximate solution: [0.5 0.5]


In [45]:
compute_error(x_ast,x)

2.4647506264441902e-12

## Método de Newton para puntos iniciales no factibles

### Punto inicial no factible ($Ax^{(0)} \neq b$)

In [46]:
x_0 = np.array([1,1],dtype=float)
nu_0 = 4

**Versión del método de Newton para puntos iniciales no factibles que se apoya del método de Newton para puntos iniciales factibles**

In [47]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast, maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.00e+00	8.49e+00	2.00e+00	1.00e+00	3.00e+00	---		1.00e+00
1	1.11e-15	1.26e-04	-1.08e-15	1.12e-15	2.44e-15	1.00e+00	1.00e+00
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
Beginning Newtons method for feasible initial point
I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.41e+00	-0.00e+00	1.12e-15	2.22e-15	---		1.00e+00
Error of x with respect to x_ast: 1.12e-15
Approximate solution: [0.5 0.5]


In [48]:
compute_error(x_ast,x)

1.1157603309187458e-15

**Versión del método de Newton para puntos iniciales no factibles (2o versión)**

In [49]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point_2nd_version(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast, maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.00e+00	8.49e+00	2.00e+00	1.00e+00	3.00e+00	---		1.00e+00
1	2.22e-16	1.26e-04	-1.91e-16	2.48e-16	6.66e-16	1.00e+00	1.00e+00
2	0.00e+00	7.85e-09	8.27e-25	5.55e-09	0.00e+00	1.00e+00	1.00e+00
Error of x with respect to x_ast: 5.55e-09
Approximate solution: [0.5 0.5]


In [50]:
compute_error(x_ast,x)

5.550929882414124e-09

# Cuarto ejemplo

$$\displaystyle \min_{x \in \mathbb{R}^2} \quad e^{x_1+3x_2-0.1} + e^{x_1 -3x_2-0.1} + e^{-x_1-0.1}$$

$$\text{sujeto a:} x_1 + 3x_2 = 0$$

In [51]:
fo = lambda x: math.exp(x[0] + 3*x[1]-0.1) + math.exp(x[0]  -3*x[1]-0.1) + math.exp(-x[0]-0.1)

In [52]:
x_ast = np.array([-0.23104907880100917,0.0770163596518852],dtype=float)

## Punto inicial $x^{(0)}$ factible ($Ax^{(0)}=b$)

In [53]:
x_0 = np.array([0,0],dtype=float)

In [54]:
A = np.array([1,3],dtype=float)

In [55]:
b=0

In [56]:
tol=1e-8
tol_backtracking=1e-14
maxiter=50
p_ast=fo(x_ast)
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_feasible_init_point(fo,A, x_0,tol, 
                                                                         tol_backtracking, x_ast, p_ast, maxiter)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	9.05e-01	1.81e-01	1.00e+00	3.81e-02	---		6.00e+00
1	9.05e-01	3.30e-03	1.34e-01	6.37e-04	1.00e+00	6.00e+00
2	9.05e-01	8.46e-07	2.15e-03	1.62e-07	1.00e+00	6.00e+00
3	9.05e-01	7.41e-14	6.52e-07	1.98e-11	1.00e+00	6.00e+00
Error of x with respect to x_ast: 6.52e-07
Approximate solution: [-0.23104893  0.07701631]


In [57]:
compute_error(x_ast, x)

6.524588707146708e-07

## Método de Newton para puntos iniciales no factibles

### Punto inicial no factible ($Ax^{(0)} \neq b$)

In [58]:
x_0 = np.array([1,1],dtype=float) #with 1,1 doesn't work
nu_0 = 1

**Versión del método de Newton para puntos iniciales no factibles que se apoya del método de Newton para puntos iniciales factibles**

In [59]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast,maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	4.00e+00	1.59e+02	1.97e+02	6.32e+00	1.81e+01	---		6.62e+02
1	4.00e+00	1.59e+02	1.97e+02	6.32e+00	1.81e+01	2.78e-17	6.62e+02
Backtracking value less than tol_backtracking, try other initial point


**Versión del método de Newton para puntos iniciales no factibles (2o versión)**

In [60]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point_2nd_version(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast,maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	4.00e+00	1.59e+02	1.97e+02	6.32e+00	1.81e+01	---		6.62e+02
1	4.00e+00	1.59e+02	1.97e+02	6.32e+00	1.81e+01	2.78e-17	6.62e+02
Error of x with respect to x_ast: 6.32e+00
Approximate solution: [1. 1.]
Backtracking value less than tol_backtracking, check approximation


### (Probando el comportamiento con punto inicial factible ($Ax^{(0)} = b$))

In [61]:
x_0 = np.array([0,0],dtype=float) 
nu_0 = 1

**Versión del método de Newton para puntos iniciales no factibles que se apoya del método de Newton para puntos iniciales factibles**

In [62]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast,maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.00e+00	3.55e+00	1.81e-01	1.00e+00	3.81e-02	---		6.00e+00
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
Beginning Newtons method for feasible initial point
I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	9.05e-01	1.81e-01	1.00e+00	3.81e-02	---		6.00e+00
1	9.05e-01	3.30e-03	1.34e-01	6.37e-04	1.00e+00	6.00e+00
2	9.05e-01	8.46e-07	2.15e-03	1.62e-07	1.00e+00	6.00e+00
3	9.05e-01	7.41e-14	6.52e-07	1.98e-11	1.00e+00	6.00e+00
Error of x with respect to x_ast: 6.52e-07
Approximate solution: [-0.23104893  0.07701631]


In [63]:
compute_error(x_ast,x)

6.524588707146708e-07

**Versión del método de Newton para puntos iniciales no factibles (2o versión)**

In [64]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point_2nd_version(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast,maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.00e+00	3.55e+00	1.81e-01	1.00e+00	3.81e-02	---		6.00e+00
1	1.11e-16	1.96e-01	3.30e-03	1.34e-01	6.37e-04	1.00e+00	6.00e+00
2	0.00e+00	3.38e-03	8.33e-07	2.14e-03	1.59e-07	1.00e+00	6.00e+00
3	0.00e+00	1.06e-06	6.98e-14	7.34e-07	1.98e-11	1.00e+00	6.00e+00
4	0.00e+00	5.92e-08	1.03e-15	1.16e-07	1.98e-11	1.00e+00	6.00e+00
5	2.78e-17	1.78e-08	5.48e-17	9.73e-08	1.98e-11	2.50e-01	6.00e+00
6	2.78e-17	1.78e-08	5.48e-17	9.73e-08	1.98e-11	2.22e-16	6.00e+00
Error of x with respect to x_ast: 9.73e-08
Approximate solution: [-0.23104906  0.07701635]
Backtracking value less than tol_backtracking, check approximation


In [65]:
compute_error(x_ast,x)

9.732233305891635e-08

### Punto inicial no factible ($Ax^{(0)} \neq b$)

In [66]:
x_0 = np.array([-.1,.1],dtype=float) 
nu_0 = 1

**Versión del método de Newton para puntos iniciales no factibles que se apoya del método de Newton para puntos iniciales factibles**

In [67]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast,maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	2.00e-01	4.81e+00	1.28e-01	5.46e-01	3.70e-02	---		6.14e+00
1	2.22e-16	5.96e-02	1.28e-05	8.39e-03	2.45e-06	1.00e+00	6.14e+00
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
Beginning Newtons method for feasible initial point
I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.06e+00	1.28e-05	8.39e-03	2.45e-06	---		5.30e+00
1	1.06e+00	1.04e-11	7.62e-06	1.78e-11	1.00e+00	5.30e+00
Error of x with respect to x_ast: 7.62e-06
Approximate solution: [-0.23104732  0.07701577]


In [68]:
compute_error(x_ast,x)

7.6180302179383765e-06

**Versión del método de Newton para puntos iniciales no factibles (2o versión)**

In [69]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point_2nd_version(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast,maxiter
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	2.00e-01	4.81e+00	1.28e-01	5.46e-01	3.70e-02	---		6.14e+00
1	2.22e-16	5.96e-02	1.28e-05	8.39e-03	2.45e-06	1.00e+00	6.14e+00
2	0.00e+00	1.34e-05	1.01e-11	7.62e-06	1.78e-11	1.00e+00	6.14e+00
3	2.78e-17	6.56e-08	1.02e-15	1.62e-07	1.98e-11	1.00e+00	6.14e+00
4	0.00e+00	6.73e-08	2.56e-16	1.25e-07	1.98e-11	5.00e-01	6.14e+00
5	2.78e-17	2.88e-08	2.66e-16	8.73e-08	1.98e-11	1.00e+00	6.14e+00
6	2.78e-17	2.88e-08	2.66e-16	8.73e-08	1.98e-11	2.22e-16	6.14e+00
Error of x with respect to x_ast: 8.73e-08
Approximate solution: [-0.23104906  0.07701635]
Backtracking value less than tol_backtracking, check approximation


In [70]:
compute_error(x_ast,x)

8.734385833741386e-08

# Quinto ejemplo: con más restricciones de igualdad

Resolver: 
$$\displaystyle \min_{x \in \mathbb{R}^3} \quad ||x||_2^2$$

$$\text{sujeto a: }\begin{array}{l}
\begin{array}{c}
x_1 + x_2 + x_3 = 1 \\
x_1 + x_2 + 2x_3 = 3
\end{array}
\end{array}
$$

In [71]:
fo = lambda x: x.dot(x)

In [72]:
x_ast = np.array([-0.5,-0.5,2. ], dtype=float)

In [73]:
A = np.array([[1, 1, 1],
              [1, 1, 2]],dtype=float)

In [74]:
b = np.array([1,3])

## Punto inicial $x^{(0)}$ factible ($Ax^{(0)}=b$)

In [75]:
x_0 = np.array([3,-4,2], dtype=float)
#x_0 = np.array([2,-3,2], dtype=float) #este es otro punto factible

In [76]:
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(x_ast)


In [77]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_feasible_init_point(fo,A, x_0,tol, 
                                                                         tol_backtracking, x_ast, p_ast, maxiter)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.08e+01	4.90e+01	2.33e+00	5.44e+00	---		1.00e+00
1	1.08e+01	3.14e-05	1.87e-03	3.48e-06	1.00e+00	1.00e+00
2	1.08e+01	5.83e-13	2.52e-07	6.45e-14	1.00e+00	1.00e+00
Error of x with respect to x_ast: 2.52e-07
Approximate solution: [-0.49999962 -0.50000038  2.        ]


In [78]:
compute_error(x_ast,x)

2.518419818612709e-07

## Método de Newton para puntos iniciales no factibles

### (Probando el comportamiento con un punto inicial factible ($Ax^{(0)} = b$))

In [79]:
x_0 = np.array([3,-4,2], dtype=float) #Ax_0 = b
#x_0 = np.array([2,-3,2], dtype=float)
nu_0 = np.array([0,0],dtype=float)

**Versión del método de Newton para puntos iniciales no factibles que se apoya del método de Newton para puntos iniciales factibles**

In [80]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.00e+00	1.08e+01	4.90e+01	2.33e+00	5.44e+00	---		1.00e+00
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
Beginning Newtons method for feasible initial point
I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	1.08e+01	4.90e+01	2.33e+00	5.44e+00	---		1.00e+00
1	1.08e+01	3.14e-05	1.87e-03	3.48e-06	1.00e+00	1.00e+00
2	1.08e+01	5.83e-13	2.52e-07	6.45e-14	1.00e+00	1.00e+00
Error of x with respect to x_ast: 2.52e-07
Approximate solution: [-0.49999962 -0.50000038  2.        ]


In [81]:
compute_error(x_ast,x)

2.518419818612709e-07

**Versión del método de Newton para puntos iniciales no factibles (2o versión)**

In [82]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point_2nd_version(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast
                                                      )

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.00e+00	1.08e+01	4.90e+01	2.33e+00	5.44e+00	---		1.00e+00
1	6.28e-16	1.01e+01	4.30e+01	2.19e+00	4.78e+00	6.25e-02	1.00e+00
2	1.26e-15	9.47e+00	3.79e+01	2.05e+00	4.21e+00	6.25e-02	1.00e+00
3	0.00e+00	8.87e+00	3.33e+01	1.92e+00	3.70e+00	6.25e-02	1.00e+00
4	6.28e-16	8.32e+00	2.92e+01	1.80e+00	3.25e+00	6.25e-02	1.00e+00
5	1.26e-15	7.80e+00	2.57e+01	1.69e+00	2.85e+00	6.25e-02	1.00e+00
6	6.28e-16	7.31e+00	2.26e+01	1.58e+00	2.51e+00	6.25e-02	1.00e+00
7	6.28e-16	6.85e+00	1.99e+01	1.49e+00	2.21e+00	6.25e-02	1.00e+00
8	6.28e-16	6.43e+00	1.74e+01	1.39e+00	1.94e+00	6.25e-02	1.00e+00
9	6.28e-16	6.02e+00	1.53e+01	1.31e+00	1.70e+00	6.25e-02	1.00e+00
10	1.26e-15	5.65e+00	1.35e+01	1.22e+00	1.50e+00	6.25e-02	1.00e+00
11	1.26e-15	5.30e+00	1.18e+01	1.15e+00	1.32e+00	6.25e-02	1.00e+00
12	6.28e-16	4.96e+00	1.04e+01	1.08e+00	1.16e+00	6.25e-02	1.00e+00
13	6.28e-16	4.65e+00	9.16e+00	1.01e+00	1.02e+00	6.25e-02	1.00e+00

In [83]:
compute_error(x_ast,x)

0.35902792568287245

**Hay que aumentar el número de iteraciones...**

### Punto inicial no factible ($Ax^{(0)} \neq b$)

In [84]:
x_0 = np.array([3,-4,0], dtype=float) #Ax_0 different to b
#x_0 = np.array([2,-3,1], dtype=float) #Ax_0 different to b
                                       #Compare different versions of
                                       #Newtons method infeasible initial points
nu_0 = np.array([0,0],dtype=float)

**Versión del método de Newton para puntos iniciales no factibles que se apoya del método de Newton para puntos iniciales factibles**

In [85]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast)

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	4.47e+00	1.00e+01	4.90e+01	2.52e+00	4.56e+00	---		1.00e+00
1	3.91e+00	8.75e+00	3.66e+01	2.20e+00	3.29e+00	1.25e-01	1.00e+00
2	3.42e+00	7.66e+00	2.73e+01	1.93e+00	2.35e+00	1.25e-01	1.00e+00
3	3.00e+00	6.70e+00	2.02e+01	1.69e+00	1.65e+00	1.25e-01	1.00e+00
4	2.62e+00	5.86e+00	1.49e+01	1.48e+00	1.13e+00	1.25e-01	1.00e+00
5	2.29e+00	5.13e+00	1.09e+01	1.29e+00	7.55e-01	1.25e-01	1.00e+00
6	2.01e+00	4.49e+00	7.89e+00	1.13e+00	4.78e-01	1.25e-01	1.00e+00
7	1.76e+00	3.93e+00	5.65e+00	9.88e-01	2.79e-01	1.25e-01	1.00e+00
8	1.54e+00	3.44e+00	3.98e+00	8.65e-01	1.37e-01	1.25e-01	1.00e+00
9	1.34e+00	3.01e+00	2.75e+00	7.57e-01	3.82e-02	1.25e-01	1.00e+00
10	1.18e+00	2.63e+00	1.84e+00	6.62e-01	2.92e-02	1.25e-01	1.00e+00
11	1.03e+00	2.30e+00	1.18e+00	5.79e-01	7.35e-02	1.25e-01	1.00e+00
12	9.01e-01	2.01e+00	7.02e-01	5.07e-01	1.01e-01	1.25e-01	1.00e+00
13	7.88e-01	1.76e+00	3.61e-01	4.44e-01	1.17e-01	1.25e-01	1.00e+00

In [86]:
compute_error(x_ast,x)

0.019618115491299666

**Hay que aumentar el número de iteraciones...**

**Versión del método de Newton para puntos iniciales no factibles (2o versión)**

In [87]:
[x,total_of_iterations,
 Err_plot,x_plot]=algorithms_for_ceco.Newtons_method_infeasible_init_point_2nd_version(fo, A, b,x_0, nu_0, tol, 
                                                       tol_backtracking, x_ast, p_ast)

I	||res_primal||	||res_dual|| 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	4.47e+00	1.00e+01	4.90e+01	2.52e+00	4.56e+00	---		1.00e+00
1	3.91e+00	8.75e+00	3.66e+01	2.20e+00	3.29e+00	1.25e-01	1.00e+00
2	3.42e+00	7.66e+00	2.73e+01	1.93e+00	2.35e+00	1.25e-01	1.00e+00
3	3.00e+00	6.70e+00	2.02e+01	1.69e+00	1.65e+00	1.25e-01	1.00e+00
4	2.62e+00	5.86e+00	1.49e+01	1.48e+00	1.13e+00	1.25e-01	1.00e+00
5	2.29e+00	5.13e+00	1.09e+01	1.29e+00	7.55e-01	1.25e-01	1.00e+00
6	2.01e+00	4.49e+00	7.89e+00	1.13e+00	4.78e-01	1.25e-01	1.00e+00
7	1.76e+00	3.93e+00	5.65e+00	9.88e-01	2.79e-01	1.25e-01	1.00e+00
8	1.54e+00	3.44e+00	3.98e+00	8.65e-01	1.37e-01	1.25e-01	1.00e+00
9	1.34e+00	3.01e+00	2.75e+00	7.57e-01	3.82e-02	1.25e-01	1.00e+00
10	1.18e+00	2.63e+00	1.84e+00	6.62e-01	2.92e-02	1.25e-01	1.00e+00
11	1.03e+00	2.30e+00	1.18e+00	5.79e-01	7.35e-02	1.25e-01	1.00e+00
12	9.01e-01	2.01e+00	7.02e-01	5.07e-01	1.01e-01	1.25e-01	1.00e+00
13	7.88e-01	1.76e+00	3.61e-01	4.44e-01	1.17e-01	1.25e-01	1.00e+00

In [88]:
compute_error(x_ast,x)

0.0523756665364094

**Hay que aumentar el número de iteraciones...**

# Comparación del quinto ejemplo con [cvxpy](https://github.com/cvxgrp/cvxpy)

In [89]:
import cvxpy as cp

In [90]:
x = cp.Variable(3)
A = np.array([[1, 1, 1],
              [1, 1, 2]])
b = np.array([1,3])

In [91]:
obj = cp.Minimize(cp.norm(x,2))

constraints = [A@x == b]

In [92]:
prob = cp.Problem(obj, constraints)

In [93]:
prob.solve()

2.1213203277662585

In [94]:
print("optimal value", prob.value)


optimal value 2.1213203277662585


In [95]:
print("optimal var", x.value)

optimal var [-0.5 -0.5  2. ]


In [96]:
A@x.value

array([1., 3.])

In [97]:
np.linalg.norm(x.value)

2.121320343558957

## Otros puntos iniciales

In [98]:
vec = [1,-2,2]
A@vec

array([1, 3])

In [99]:
np.linalg.norm(vec)

3.0

In [100]:
vec = [2,-3,2]
A@vec

array([1, 3])

In [101]:
np.linalg.norm(vec)

4.123105625617661

In [102]:
vec = [3,-4,2]
A@vec

array([1, 3])

In [103]:
np.linalg.norm(vec)

5.385164807134504

**Referencias:**

* S. P. Boyd, L. Vandenberghe, Convex Optimization, Cambridge University Press, 2009.